### Danny Ma 8 week SQL course

Introduction
Danny seriously loves Japanese food so in the beginning of 2021, he decides to embark upon a risky venture and opens up a cute little restaurant that sells his 3 favourite foods: sushi, curry and ramen.

Danny’s Diner is in need of your assistance to help the restaurant stay afloat - the restaurant has captured some very basic data from their few months of operation but have no idea how to use their data to help them run the business.

Problem Statement
Danny wants to use the data to answer a few simple questions about his customers, especially about their visiting patterns, how much money they’ve spent and also which menu items are their favourite. Having this deeper connection with his customers will help him deliver a better and more personalised experience for his loyal customers.

He plans on using these insights to help him decide whether he should expand the existing customer loyalty program - additionally he needs help to generate some basic datasets so his team can easily inspect the data without needing to use SQL.

Danny has provided you with a sample of his overall customer data due to privacy issues - but he hopes that these examples are enough for you to write fully functioning SQL queries to help him answer his questions!

Danny has shared with you 3 key datasets for this case study:
<br /> - sales
<br /> - menu
<br /> - members
<br />You can inspect the entity relationship diagram and example data below.

![alt text](tableERD.png "ERD Diagram")

<u>Example Datasets</u>
<br /> All datasets exist within the dannys_diner database schema - be sure to include this reference within your SQL scripts as you start exploring the data and answering the case study questions.

<b>Table 1: sales </b>
<br />The sales table captures all customer_id level purchases with an corresponding order_date and product_id information for when and what menu items were ordered.

![alt text](sales.png "Sales Table")

<b>Table 2: menu</b><br />
The menu table maps the product_id to the actual product_name and price of each menu item.

![alt text](menu.png "Menu Table")

<b>Table 3: members</b> <br />
The final members table captures the join_date when a customer_id joined the beta version of the Danny’s Diner loyalty program.

![alt text](members.png "Members Table")

<b>Case Study Questions</b> <br />
Each of the following case study questions can be answered using a single SQL statement:

1. What is the total amount each customer spent at the restaurant?

<br /> SELECT customer_id, sum(price) as spent
<br />FROM dannys_diner.sales s
<br /> left join dannys_diner.menu m
<br /> on s.product_id=m.product_id
<br /> group by customer_id
<br /> order by customer_id;

![alt text](q1.png "question 1")

2. How many days has each customer visited the restaurant?

<br />select customer_id, count(*) as days_visited
<br />from
<br />(SELECT customer_id, order_date
<br />FROM dannys_diner.sales 
<br />group by customer_id, order_date) base_table
<br />group by customer_id
<br />order by customer_id

![alt text](q2.png "question 2")

3. What was the first item from the menu purchased by each customer?
<br />Customer A purchased sushi and ramen on their first visit

<br /> select customer_id, product_name as first_item
<br /> from
<br /> (SELECT customer_id, product_name, rank() over (partition by customer_id order by order_date) as ranking
<br /> FROM dannys_diner.sales s
<br /> left join dannys_diner.menu m 
<br /> on s.product_id = m.product_id 
<br /> group by customer_id, product_name, order_date) base_table
<br /> where ranking = 1

![alt text](q3.png "question 3")

4. What is the most purchased item on the menu and how many times was it purchased by all customers?

<br /> select product_name, times_ordered 
<br /> from
<br /> (select product_name, times_ordered, rank() over (order by times_ordered desc) as ranking
<br /> from
<br /> (SELECT product_name, count(*) as times_ordered
<br /> FROM dannys_diner.sales s
<br /> left join dannys_diner.menu m 
<br /> on s.product_id = m.product_id 
<br /> group by product_name) base_table
<br /> ) basetable2
<br /> where ranking = 1

![alt text](q4.png "question 4")

5. Which item was the most popular for each customer?
<br /> customer B has tried all the items the same amt of times

<br /> select customer_id, product_name, times_ordered 
<br /> from
<br /> (select customer_id, product_name, times_ordered, rank() over (partition by customer_id order by times_ordered desc) as ranking
<br /> from
<br /> (SELECT customer_id, product_name, count(*) as times_ordered
<br /> FROM dannys_diner.sales s
<br /> left join dannys_diner.menu m 
<br /> on s.product_id = m.product_id 
<br /> group by customer_id, product_name) base_table
<br /> ) basetable2
<br /> where ranking = 1

![alt text](q5.png "question 5")

6. Which item was purchased first by the customer after they became a member?
<br /> Here I am giving this answer as cust A ordered curry on 1/7 and became a member on the same day. Because the question does not state that it has to be the order after the date they become a member - I give this answer. The timestamp is the exact same - so I do not know the customer did not become a member and then order 

<br />select customer_id, product_name
<br />from
<br />(select *, rank() over (partition by customer_id order by ranking ) as ranking1
<br />from
<br />(SELECT s.customer_id, product_name, case when join_date <= order_date then rank() over (partition by s.customer_id order by order_date)  end as ranking, join_date, order_date
<br />FROM dannys_diner.sales s
<br />left join dannys_diner.menu m 
<br />on s.product_id = m.product_id 
<br />left join dannys_diner.members me
<br />on me.customer_id=s.customer_id
<br />group by s.customer_id, product_name, order_date, join_date) base_table
<br />where ranking is not null) base2
<br />where ranking1= 1

![alt text](q6.png "question 6")

7. Which item was purchased just before the customer became a member?
<br /> cust A only has the one order before they become a member on 1/7 and it's their first order

<br />select customer_id, product_name
<br />from
<br />(select *, rank() over (partition by customer_id order by order_date desc) as ranking1
<br />from
<br />(SELECT s.customer_id, product_name, case when join_date > order_date then rank() over (partition by s.customer_id order by order_date)  end as ranking, join_date, order_date
<br />FROM dannys_diner.sales s
<br />left join dannys_diner.menu m 
<br />on s.product_id = m.product_id 
<br />left join dannys_diner.members me
<br />on me.customer_id=s.customer_id
<br />group by s.customer_id, product_name, order_date, join_date) base_table
<br />where ranking is not null) base2
<br />where ranking =1

![alt text](q7.png "question 7")

8. What is the total items and amount spent for each member before they became a member?

<br />select customer_id, count(*) as items, sum(price) as spend
<br />from
<br />(SELECT s.*, product_name, case when join_date > order_date or join_date is null then 1 else 0 end as non_mem, join_date,  price
<br />FROM dannys_diner.sales s
<br />left join dannys_diner.menu m
<br />on s.product_id = m.product_id
<br />left join dannys_diner.members me
<br />on me.customer_id=s.customer_id
<br />order by customer_id,order_date) base_table
<br />where non_mem = 1
<br />group by customer_id

![alt text](q8.png "question 8")

9. If each $1 spent equates to 10 points and sushi has a 2x points multiplier - how many points would each customer have?

<br />select customer_id, sum(points) total_points
<br />from
<br />(select s.*, product_name,
<br />case when product_name = 'sushi' then (price * 20) else (price * 10) end as points
<br />FROM dannys_diner.sales s
<br />left join dannys_diner.menu m
<br />on s.product_id=m.product_id
<br />order by customer_id, order_date) inner_table
<br />group by customer_id

![alt text](q9.png "question 9")

10. In the first week after a customer joins the program (including their join date) they earn 2x points on all items, not just sushi - how many points do customer A and B have at the end of January?
<br /> here I get an answer different than a lot of the others I see people with online - but that is due to the fact that as customer A became a member on 1/7 and ordered on 1/7 - I include that order towards their points as a member

<br />select customer_id, sum(pts_jan) as pts_jan
<br />from
<br />(select *, case when order_date <= '2021-01-31' then pts else 0 end as pts_jan
<br />from
<br />(SELECT s.*, join_date, 
<br />case when join_date-order_date between -6 and 0  then price*20 else case when product_name = 'sushi' then (price * 20) else (price * 10) end end as pts 
<br />FROM dannys_diner.sales s
<br />left join dannys_diner.members me
<br />on me.customer_id=s.customer_id
<br />left join dannys_diner.menu m
<br />on m.product_id = s.product_id) inner_table
<br />) base2
<br />where join_date is not null
<br />group by customer_id

![alt text](q10.png "question 10")

<b>Join All The Things</b><br />
The following questions are related creating basic data tables that Danny and his team can use to quickly derive insights without needing to join the underlying tables using SQL.

Recreate the following table output using the available data:

![alt text](joined.png "Joined Table")

    SELECT
      	s.customer_id,
        left(cast(order_date as text),10) order_date,
        product_name,
        price,
        case when order_date >= join_date then 'Y' else 'N' end as member
    from dannys_diner.sales s     
    left join dannys_diner.members me
    on s.customer_id = me.customer_id
    left join dannys_diner.menu m 
    on s.product_id = m.product_id
    order by customer_id, order_date, s.product_id;

| customer_id | order_date | product_name | price | member |
| ----------- | ---------- | ------------ | ----- | ------ |
| A           | 2021-01-01 | sushi        | 10    | N      |
| A           | 2021-01-01 | curry        | 15    | N      |
| A           | 2021-01-07 | curry        | 15    | Y      |
| A           | 2021-01-10 | ramen        | 12    | Y      |
| A           | 2021-01-11 | ramen        | 12    | Y      |
| A           | 2021-01-11 | ramen        | 12    | Y      |
| B           | 2021-01-01 | curry        | 15    | N      |
| B           | 2021-01-02 | curry        | 15    | N      |
| B           | 2021-01-04 | sushi        | 10    | N      |
| B           | 2021-01-11 | sushi        | 10    | Y      |
| B           | 2021-01-16 | ramen        | 12    | Y      |
| B           | 2021-02-01 | ramen        | 12    | Y      |
| C           | 2021-01-01 | ramen        | 12    | N      |
| C           | 2021-01-01 | ramen        | 12    | N      |
| C           | 2021-01-07 | ramen        | 12    | N      |

<b>Rank All The Things</b><br />
Danny also requires further information about the ranking of customer products, but he purposely does not need the ranking for non-member purchases so he expects null ranking values for the records when customers are not yet part of the loyalty program.

![alt text](ranking.png "Ranked Table")

    select *, 
     case when member ='Y' then dense_rank() over (partition by customer_id, member order by order_date)  end  as ranking
    from
    (SELECT
      	s.customer_id,
        left(cast(order_date as text),10) order_date,
        product_name,
        price,
        case when order_date >= join_date then 'Y' else 'N' end as member
    from dannys_diner.sales s     
    left join dannys_diner.members me
    on s.customer_id = me.customer_id
    left join dannys_diner.menu m 
    on s.product_id = m.product_id
    order by customer_id, order_date, s.product_id) base_table;

| customer_id | order_date | product_name | price | member | ranking |
| ----------- | ---------- | ------------ | ----- | ------ | ------- |
| A           | 2021-01-01 | sushi        | 10    | N      |         |
| A           | 2021-01-01 | curry        | 15    | N      |         |
| A           | 2021-01-07 | curry        | 15    | Y      | 1       |
| A           | 2021-01-10 | ramen        | 12    | Y      | 2       |
| A           | 2021-01-11 | ramen        | 12    | Y      | 3       |
| A           | 2021-01-11 | ramen        | 12    | Y      | 3       |
| B           | 2021-01-01 | curry        | 15    | N      |         |
| B           | 2021-01-02 | curry        | 15    | N      |         |
| B           | 2021-01-04 | sushi        | 10    | N      |         |
| B           | 2021-01-11 | sushi        | 10    | Y      | 1       |
| B           | 2021-01-16 | ramen        | 12    | Y      | 2       |
| B           | 2021-02-01 | ramen        | 12    | Y      | 3       |
| C           | 2021-01-01 | ramen        | 12    | N      |         |
| C           | 2021-01-01 | ramen        | 12    | N      |         |
| C           | 2021-01-07 | ramen        | 12    | N      |         |